#### A program for comparing the difference between FWHM before and after deconvolution with three different algorithms for a range of generated Voigt functions

In [1]:
%matplotlib qt

from scipy.special import wofz
import numpy as np
import hyperspy.api as hs
from ncempy.io import dm
import matplotlib.pyplot as plt
from scipy.signal import peak_widths, find_peaks
import math
from tabulate import tabulate

Voigt Function Generator

In [2]:
def G(x, alpha):
    """ Return Gaussian line shape at x with HWHM alpha """
    return np.sqrt(np.log(2) / np.pi) / alpha\
                             * np.exp(-(x / alpha)**2 * np.log(2))

def L(x, gamma):
    """ Return Lorentzian line shape at x with HWHM gamma """
    return gamma / np.pi / (x**2 + gamma**2)

def V(x, alpha, gamma):
    """
    Return the Voigt line shape at x with Lorentzian component HWHM gamma
    and Gaussian component HWHM alpha.

    """
    sigma = alpha / np.sqrt(2 * np.log(2))

    return np.real(wofz((x + 1j*gamma)/sigma/np.sqrt(2))) / sigma\
                                                           /np.sqrt(2*np.pi)

Functions

In [3]:
#plot the spectrum with HyperSpy
def hyperspy_plot(ev, counts):
    s = hs.signals.EELSSpectrum(counts)
    s.axes_manager[0].scale = np.diff(ev).mean()
    s.axes_manager[0].unit = 'eV'
    s.axes_manager[0].offset = ev[0]
    s.axes_manager[0].name = 'Energy'
    return s

#FWHM comparisons
def FWHM_testing(alpha, gamma, hs_signal, hs_deconvolved, height):
    
    peaks1, _ = find_peaks(hs_signal, height=0.5)
    results_half_signal = peak_widths(hs_signal, peaks1, rel_height=0.5)
    
    peaks2, _ = find_peaks(hs_deconvolved, height=height)
    results_half_deconvolved = peak_widths(hs_deconvolved, peaks2, rel_height=0.5)
    
    FWHM_signal = 4 / 1000 * results_half_signal[0]
    FWHM_deconvolved = 4 / 1000 * results_half_deconvolved[0]
    
    Lorentzian_FWHM = 2 * gamma
    Gaussian_FWHM = 2 * alpha
    
    relative_error = abs((FWHM_deconvolved[0] - Lorentzian_FWHM)/Lorentzian_FWHM*100)
    return math.trunc(relative_error)

### Richardson Lucy

In [4]:
#Richardson-Lucy algorithm (code from Edson Bellido)
def RL(iterations, PSF, Spectrum):
    RL4 = np.copy(Spectrum)
    for i in range(iterations):
        RL1 = np.convolve(PSF, RL4, mode='same')
        RL2 = np.divide(Spectrum,RL1)
        RL3 = np.convolve(PSF, RL2, mode='same')
        RL4 = np.multiply(RL3, RL4)
    return RL4

In [5]:
def voigt_RL(iterations):
    lst = [0.1, 0.2, 0.3, 0.4, 0.5]
    x = np.linspace(-2,2,1000)
    data = []
    
    for alpha in lst:
        dt = [alpha]
        gaussian = G(x, alpha)
        for gamma in lst:
            voigt = V(x, alpha, gamma)
            s_signal = hyperspy_plot(x, voigt)
            RL_deconvolve = RL(iterations, gaussian, voigt)
            s_RL = hyperspy_plot(x, RL_deconvolve)
            error = FWHM_testing(alpha, gamma, s_signal, s_RL, 0.5)
            dt.append(error)
        data.append(dt)
    print(tabulate(data, headers=['alpha\gamma', '0.1', '0.2', '0.3', '0.4', '0.5'], tablefmt="plain"))

In [6]:
voigt_RL(1000)

  alpha\gamma    0.1    0.2    0.3    0.4    0.5
          0.1      2      0      0      0      0
          0.2     22      2      1      0      4
          0.3     50     14     11     12     93
          0.4     62      3     19     34     90
          0.5    112     73     51     82     86


### MEM by Meinel

In [7]:
def Meinel(iterations, PSF, Spectrum):
    MEM = np.copy(Spectrum)
    for i in range(iterations):
        MEM1 = np.convolve(PSF, MEM, mode='same')
        MEM2 = np.divide(Spectrum, MEM1)
        MEM3 = np.multiply(MEM2, MEM2)
        MEM4 = np.convolve(PSF, MEM3, mode='same')
        MEM5 = np.multiply(MEM4, MEM)
        MEM = MEM5
    return MEM

In [8]:
def voigt_Meinel(iterations):
    lst = [0.1, 0.2, 0.3, 0.4, 0.5]
    x = np.linspace(-2,2,1000)
    data = []
    
    for alpha in lst:
        dt = [alpha]
        gaussian = G(x, alpha)
        for gamma in lst:
            voigt = V(x, alpha, gamma)
            s_signal = hyperspy_plot(x, voigt)
            MEM_deconvolve = Meinel(iterations, gaussian, voigt)
            s_MEM = hyperspy_plot(x, MEM_deconvolve)
            error = FWHM_testing(alpha, gamma, s_signal, s_MEM, 0.5)
            dt.append(error)
        data.append(dt)
    print(tabulate(data, headers=['alpha\gamma', '0.1', '0.2', '0.3', '0.4', '0.5'], tablefmt="plain"))

In [9]:
voigt_Meinel(1000)

  alpha\gamma    0.1    0.2    0.3    0.4    0.5
          0.1      1      0      0      0     98
          0.2     20      3      1      1     96
          0.3     38      0     12     11     94
          0.4     52      2     25     39     91
          0.5    131    100     67     84     88


### MEM by Carasso

In [10]:
def Carasso(iterations, rho, PSF, Spectrum):
    C = rho
    N = np.sum(Spectrum)
    
    MEM = Spectrum
    for i in range(iterations):
        A1 = np.convolve(PSF, MEM, mode='same')
        A2 = np.divide(Spectrum, A1)
        A3 = np.convolve(PSF, A2, mode='same')
        A4 = np.subtract(np.multiply(rho, A3), rho)
        A5 = np.add(np.subtract(A4, np.log10(MEM)), C)
        A6 = N * (np.sum(np.multiply(MEM, A5)))**(-1)
        
        MEM1 = np.convolve(PSF, MEM, mode='same')
        MEM2 = np.divide(Spectrum, MEM1)
        MEM3 = np.convolve(PSF, MEM2, mode='same')
        MEM4 = np.subtract(np.multiply(rho, MEM3), rho)
        MEM5 = np.add(np.subtract(MEM4, np.log10(MEM)), C)
        MEM6 = np.multiply(np.multiply(MEM, MEM5), A6)
        MEM = MEM6
    return MEM

In [12]:
def voigt_Carasso(iterations, rho):
    lst = [0.1, 0.2, 0.3, 0.4, 0.5]
    x = np.linspace(-2,2,1000)
    data = []
    
    for alpha in lst:
        dt = [alpha]
        gaussian = G(x, alpha)
        for gamma in lst:
            voigt = V(x, alpha, gamma)
            s_signal = hyperspy_plot(x, voigt)
            MEM_deconvolve = Carasso(iterations, rho, gaussian, voigt)
            s_MEM = hyperspy_plot(x, MEM_deconvolve)
            error = FWHM_testing(alpha, gamma, s_signal, s_MEM, 0.5)
            dt.append(error)
        data.append(dt)
    print(tabulate(data, headers=['alpha\gamma', '0.1', '0.2', '0.3', '0.4', '0.5'], tablefmt="plain"))

In [14]:
voigt_Carasso(1000, 1000)

  alpha\gamma    0.1    0.2    0.3    0.4    0.5
          0.1      2      0      0      0      0
          0.2     24      2      0      0      3
          0.3     53     16     11     10      4
          0.4     69      9     13     27     89
          0.5    115     69     50     26     85
